# Wikipedia Retrieval with Claude

This notebook provides a step-by-step guide for using the Wikipedia search tool with Claude. We will:

1. Set up the environment and imports
2. Build a search tool to query the Wikipedia API
3. Test the search tool  
4. Create a Claude client with access to the tool 
5. Compare Claude's responses with and without access to the tool

## Imports and Configuration 

First we'll import libraries and load environment variables. This includes setting up logging so we can monitor the process.

In [ ]:
import os
import sys
import dotenv
import anthropic

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

import claude_retriever

# Load environment variables
dotenv.load_dotenv()

In [ ]:
# Import and configure logging 
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a handler to log to stdout
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(handler)

## Make a Wikipedia Search Tool

After importing the WikipediaSearchTool class, we can easily initalize a new WikipediaSearchTool object.

Let's test the search tool to make sure it works:

In [ ]:
from claude_retriever.searcher.searchtools.wikipedia import WikipediaSearchTool

# Create a searcher
wikipedia_search_tool = WikipediaSearchTool()

# Run a test query
query = "LK-99"

results = wikipedia_search_tool.search(query, n_search_results_to_use=1)
print(results)

## Use Claude with Retrieval

Now we simply pass the search tool to a Claude client, allowing Claude access to use it when generating responses. 

In [ ]:
ANTHROPIC_SEARCH_MODEL = "claude-2"

client = claude_retriever.ClientWithRetrieval(api_key=os.environ['ANTHROPIC_API_KEY'], verbose=True, search_tool = wikipedia_search_tool)

query = "What's the name of the latest material that was claimed to be a room temperature superconductor?"
prompt = f'{anthropic.HUMAN_PROMPT} {query}{anthropic.AI_PROMPT}'

Here is the basic response to the query (no access to the tool).

In [ ]:
basic_response = client.completions.create(
    prompt=prompt,
    stop_sequences=[anthropic.HUMAN_PROMPT],
    model=ANTHROPIC_SEARCH_MODEL,
    max_tokens_to_sample=1000,
)
print('-'*50)
print('Basic response:')
print(prompt + basic_response.completion)
print('-'*50)


Now let's try getting a completion from Claude equipped with the Wikipedia search tool.

In [ ]:
augmented_response = client.completion_with_retrieval(
    query=query,
    model=ANTHROPIC_SEARCH_MODEL,
    n_search_results_to_use=1,
    max_searches_to_try=3,
    max_tokens_to_sample=1000)

print('-'*50)
print('Augmented response:')
print(prompt + augmented_response)
print('-'*50)